<a href="https://colab.research.google.com/github/slachitoff/CS-GY-6613-Assignments/blob/main/CS_GS_6613_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1
Deep-SORT takes sequential data, in this case video frames, as it's initial input. Those images are passed into a Faster Region-based Convolutional Neural Network (FrRCNN) which acts as an object detector. 

The FrRCNN is itself a multi-stage process, beginning with feature extraction. Here, each image is passed through a CNN backbone which generates a high-dimensional vector of features. This vector is then passed onto the next stage of the Object Detection process, the Region Proposal Network (RPN), which uses anchor windows to generate a set of Regions of Interest (RoIs), or areas in the image that are likely to contain objects of significance. This set of proposals is then sent to the RoI Pooling layer, which uses the object feature map generated in the feature extraction stage to interpolate the features of the object proposals into a fixed-size feature map. This fixed-size feature map is then fed into a classification network which outputs detections in the form of class labels, confidence scores, and bounding box locations for each proposal.

In the tracking stage, a Kalman filter uses the object features, spatial features, and detections output from the object detection stage to predict the future state of each target.  These state predictions are used to propagate object identities across frames, as well as maintain object identities in the event of occlusion..  

The state of each object is represented as an eight-dimensional vector $x = [u, v, \gamma, h, \dot{u}, \dot{v}, \dot{\gamma}, \dot{h}]^T$, where $u$ and $v$ are the horizontal and vertical coordinates of the center of the target object, $\gamma$ is the aspect ratio of the bounding box, $h$ is the height of the bounding box, and $\dot{u}, \dot{v}, \dot{\gamma}, \dot{h}$ represent the respective velocities of the horizontal, vertical, aspect ratio, and height dimensions.

A linear constant velocity model $x_t=A_tx_{t-1} + w_t$ is used to predict the state $x$ of each object at frame $t$, where $x_{t-1}$ is the state of the object in the previous frame. Here, $A_t$ is a $8 \times 8$ state transition matrix which describes how the state of the object has changed from frame $t-1$ to frame $t$.

When the true location of the object is determined in later frames, it is input back into the Kalman filter as an observation. This observation is expressed as $z_t = C_t x_t+v_t$, where $C_t$ is a $4 \times 8$ matrix that maps the state $x_t$ to a measurement space of size $4$ (for the $u$, $v$, $\gamma$, and $h$ coordinates), and $v_t$ is the measurement noise representing the degree of uncertainty regarding the measurement $z_t$.

The Kalman Gain at time $t$ can be expressed as $K_t = \hat{\Sigma_t} C_t^T (C_t \hat{\Sigma_t} C_t^T + Q_t)^{-1}$, where $\hat{\Sigma_t}$ is the predicted state covariance matrix, $C_t$ is the measurement matrix, and $Q_t$ is the measurement noise covariance matrix.

The Kalman Gain and the measurement are used to derive $x_t = \hat{x}_t + K_t(z_t-C_t\hat{x}_t)$, where $\hat{x}_t$ is the predicted state estimate, and $C_t\hat{x}_t$ is the predicted measurement. The state covariance matrix is also updated, giving $\Sigma_t = (I - K_t C_t)\hat{\Sigma_t}$.

These measurements are used to decrease the probability distribution variance, thereby improving the accuracy of subsequent predictions. At each frame $t$, the Kalman filter outputs a set of predictions regarding the object states, $\hat{x}_t$, along with the associated state covariance matrices $\Sigma_t$.

The object features, spatial features, and detections in the object detection stage, as well as the predicted object states output by the the Kalman filter, are used as input into the data association stage of the process. 

The probability distributions output by the Kalman filter are compared to the observed positions of the detected objects in each frame forming what is called the Mahalanobis distance.

As part of the data association stage, the deep appearance descriptor is a CNN which is trained on a large-scale person re-identification dataset. It takes as input a cropped image of an objects outputs a high-dimensional feature vector that represents the appearance of the object in the image. The cosine distance between vectors can then be computed, resulting in a score representing the similarity between the appearance of objects between frames.

The resulting cosine distance score from the deep appearance descriptor, and the Mahalanobis distance can be linearly combined to form a scalar value that can be used by the Hungarian algorithm to assign detections to their identities.

The Hungarian algorithm is a global optimization algorithm which takes as its input a cost matrix consisting of the intersection-over-union (IoU) distances between the bounding box locations output by the object detection classifier network, and the predicted states produced by the Kalman filter. Given $n$ targets, we have $n$ predicted states which must be matched to $n$ detections.

A brute force approach to this linear assignment problem is of $\theta(n!)$ time complexity. In comparison, the Hungarian algorithm is able to solve this same problem in $\theta(n^3)$, thereby establishing an initial correspondence between the tracks and the detections more efficiently than otherwise possible.

Cascade matching is an extension of the IoU matching algorithm mentioned above. In an effort to reduce the time complexity of the Hungarian algorithm, cascade matching attempts to match the most recent detections to the most recent identifications, and older detections with older identifications. 











# Task 3
In the original Simple Online and Realtime Tracking paper, the authors explicitly state that, "...incorporating complexity in the form of object re-identification adds significant overhead..." which could potentially limit the abilit for SORT to be used in real-time applications. Later in the paper, they reiterate this shortcoming, stating that, "...object re-identification is beyond the scope of
this work..." That being said, the follow-up, Simple Online and Realtime Tracking With a Deep Association Metric, attempts to address this issue by introducing the Deep Appearance Descriptor which is explicitly trained on a re-identification dataset, in an effort to reduce the likelihood of track fragmentation or identity switches. Even so, there are limitations, and given that in a game of soccer, players are wearing nearly identical outfits, much of the discriminative power of their appearance have been greatly reduced. The smaller details such as facial features are far less apparent, and thus is is completely understandable as to why the Deep-SORT algorithm would struggle to identify players.